# Primary data analysis: transactions

Transaction history consists of 14 columns, which are described by the table below. In this notebook, an exploratory data analysis is performed on all columns.

| Column name          | Description                                      | Values |
|----------------------|--------------------------------------------------| ------ |
| card_id              | Card identifier                                  |  325540 unique (total: 29112361) |
| month_lag            | month lag to reference date                      | Number between -4.5 and -1.3 |
| purchase_date        | Purchase date                                    | Y-m-d H:m:s |
| authorized_flag      | 'Y' if approved, 'N' if denied                   | Y / N |
| category_3           | anonymised category                              | A / B / C / nan |
| installments         | number of installments of purchase               | Number between -1 and 999 |
| category_1           | anonymised category                              | Y / N |
| merchant_category_id | Merchant category identifier (anonymised)        | 327 unique values |
| subsector_id         | Merchant category group identifier (anonymised)  | 41 unique values |
| merchant_id          | Merchant identifier (anonymised)                 | 326312 unique values |
| purchase_amount      | Normalized purchase amount                       | Number between -7.5 and 6 |
| city_id              | City identifier (anonymised)                     | 308 unique values |
| state_id             | State identifier (anonymised)                    | 25 unique values |
| category_2           | anonymised category                              | 1 / 2 / 3 / 4 / 5 / nan |

## Suggested column features
All data should be grouped by `card_id` while creating features.

### [card_id](#card_id-exploration)
- Key
- Count of transactions per card id.

### [category_n](#category-exploration)
- Fraction of total products bought in category within `category_n`
- Total bought in category within `category_n`.
- Purchase amount per category.
- Purchase counts per category.

### [month_lag](#month_lag-exploration)
- Average
- Count
- Stddev
- Max
- Min

### [purchase_date](#purchase_date-exploration)
- Day of week
- Day of month
- Month in year
- **EXTERNAL DATA**: Season

### [authorized_flag](#authorized_flag-exploration)
- Count `Y` and `N` values.
- Fraction `Y / N`
- Becomes a cross column with other features

### [installments](#installments-exploration)
- Filter out outlier(s)
- Total installments of card id

### [purchase_amount](#purchase_amount-exploration)
- Filter out extremely high values or give them a maximum value of - say - 10.
- Log1p(?)

### [city_id](#city_id-exploration)
- Most dominant (max purchase amount)
- Fraction of purchase amount per city per card id
- Total, average, stddev purchase amount
- Positive purchase amount
- Negative purchase amount

### [state_id](#state_id-exploration)
- Most dominant (max purchase amount)
- Fraction of purchase amount per state per card id
- Total, average, stddev purchase amount
- Positive purchase amount
- Negative purchase amount

### [merchant_category_id](#merchant_category_id-exploration)
- Most dominant (max purchase amount)
- _Skip_

### [subsector_id](#subsector_id-exploration)
- Most dominant (max purchase amount)
- _Skip_

### [merchant_id](#merchant_id-exploration)
- Most dominant (max purchase amount)
- _Skip_

The `new_merchant_transactions.csv` file can be added to the historical transactions file with a flag indicating the 'new merchant'. The indicator must be used in combination with a cross column (just like with the `authorized flag`).

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pprint import pprint

# Append PATH, such that methods defined in the notebooks folder can be loaded.
sys.path.append('./notebooks')

import seaborn as sns
sns.set(color_codes=True)

# # Import Dask, which allows for parallel data processing in a Pandas-way.
# import dask.dataframe as dd
# import dask.array as da
# from dask.array import stats

# Use pandas, which runs fast with a relatively small dataset (compared to Dask).
import pandas as pd

import numpy as np

from column_explore import dataframe_explore

In [ ]:
# transactions = dd.read_csv('../data/raw/historical_transactions.csv')  # Use Dask
transactions = pd.read_csv('../data/raw/historical_transactions.csv')  # Use Pandas

In [ ]:
transactions.head()

In [ ]:
transactions_histogram = transactions.hist(figsize=(15,10))

## Primary column exploration

In [ ]:
# Skip the fundamental data exploration as it takes a lot of time and provides too
# little information.
# fundamental_details = dataframe_explore(transactions)

# card_id exploration
Observations:

- Some ID's have thousands of purchases.
- The `log1p` of the purchase counts seems normally distributed (whatever that means).

In [ ]:
# Determine frequencies of card_id's.
card_id_frequencies = transactions.groupby('card_id').agg('count')['authorized_flag']  # Picking a random column to determine the counts.

In [ ]:
card_id_frequencies.authorized_flag.sort_values(ascending=False).head(20)

In [ ]:
card_id_histogram = sns.distplot(np.log1p(card_id_frequencies.authorized_flag.values), bins=50)

# category exploration

Observations:
The categories are distributed as follows:
    
**category_1**

| value | fraction of data |
| ---- | ---- |
| N | 0.928414 |
| Y | 0.071586 |

**category_2**

| value  | fraction of data |
| ------ | ---------------- |
| 1.0    | 0.573601         |
| 2.0    | 0.038796         |
| 3.0    | 0.147841         |
| 4.0    | 0.098946         |
| 5.0    | 0.140816         |

**category_3**

| value  | fraction of data |
| ------ | ---------------- |
|   A    | 0.532648         |
|   B    | 0.403589         |
|   C    | 0.063763         |


In [ ]:
# category_1_frequencies = transactions[['category_1', 'city_id']].groupby(['category_1'])['city_id'].count().compute()
# category_2_frequencies = transactions[['category_2', 'city_id']].groupby(['category_2'])['city_id'].count().compute()
# category_3_frequencies = transactions[['category_3', 'city_id']].groupby(['category_3'])['city_id'].count().compute()

# pprint(category_1_frequencies / category_1_frequencies.sum())
# pprint(category_2_frequencies / category_2_frequencies.sum())
# pprint(category_3_frequencies / category_3_frequencies.sum())

# purchase_amount exploration
Observations:

- Neither the log1p, nor the regular histogram show a normal distribution of the purchase amount.
- Most of the values lie between -0.75 and 0. There are some extreme values, up to millions.
- There are also negative values in the column.

In [ ]:
transactions_histogram = transactions[transactions['purchase_amount'] < 1]['purchase_amount'].hist(bins=10)

# month_lag exploration
Observations:

- Varies between 0 and -13.
- 75% of all values are >= -2.

In [ ]:
transactions.month_lag.describe()

In [ ]:
month_lag_plot = transactions  \
                    .groupby('month_lag')  \
                    .agg({'card_id': 'nunique'})  \
                    .reset_index('month_lag')  \
                    .plot  \
                    .scatter(x='month_lag', y='card_id')

# installments exploration
Observations:

- There's an outlier with a number of installments of 999.
- Some card id's show a number of installments of <0.
- Most card id's have 1 installment.

In [ ]:
transactions.installments.describe()

In [ ]:
installments_histogram = sns.distplot(transactions[transactions.installments < 30].installments.values, bins=50)

# purchase_date exploration
Observations:

_None_

In [ ]:
transactions.purchase_date.describe()

# city_id exploration
Observations:

- There are 308 different cities.
- The _smallest_ city (185) has 20 customers; the largest (69) almost 250k.
- There are nearly 5.2k card_id's on average per city.

In [ ]:
transactions.groupby('city_id').agg({'card_id': ['nunique']}).describe()

# state_id exploration
Observations:
- There are 25 states in which there are transactions.
- The average number of card_id's per state is 35.6k.
- The minimal number of card_id's in a state is 490; the maximum 270k.

In [ ]:
transactions.groupby('state_id').agg({'card_id': ['nunique']}).describe()

# merchant_category_id exploration
Observations:

- There are 327 merchant categories.
- The average number of card_id's per merchant category id is 18k.
- The largest merchant category id was bought by 285k customers.

In [ ]:
transactions.groupby('merchant_category_id').agg({'card_id': ['nunique']}).describe()

# subsector_id exploration
Observations:

- There are 41 subsector id's.
- Per subsector id, there are 91k customer id's on average.
- The smallest subsector id was bought by 80 customers.

In [ ]:
transactions.groupby('subsector_id').agg({'card_id': ['nunique']}).describe()

# merchant_id exploration
Observations:

- There are 326,311 merchants.
- There are 34 customers per merchant id on average.
- The highest number of customers of a single merchant is 216k.

In [ ]:
transactions.groupby('merchant_id').agg({'card_id': ['nunique']}).describe()